In [2]:
import tensorflow as tf
from tensorflow import keras
import sklearn

import numpy as np
import pandas as pd
import os

np.random.seed(42)
tf.random.set_seed(42)

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
plt.style.use('seaborn-darkgrid')

# Tensor

### tensor & arithmetic operation

In [3]:
print(tf.constant([[1.,2.,3.], [4.,5.,6.]])) # matrix
print(tf.constant(42)) # scalar

tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]], shape=(2, 3), dtype=float32)
tf.Tensor(42, shape=(), dtype=int32)


In [5]:
t = tf.constant([[1.,2.,3.], [4.,5.,6.]])
print(t.shape)
print(t.dtype)

(2, 3)
<dtype: 'float32'>


In [12]:
print(t[:,1:])
print(t[...,1])
print(t[...,1,tf.newaxis])

tf.Tensor(
[[2. 3.]
 [5. 6.]], shape=(2, 2), dtype=float32)
tf.Tensor([2. 5.], shape=(2,), dtype=float32)
tf.Tensor(
[[2.]
 [5.]], shape=(2, 1), dtype=float32)


In [16]:
print(t+10)
print(t-1)
print(t*2)
print(tf.square(t))
print(t@tf.transpose(t))

tf.Tensor(
[[11. 12. 13.]
 [14. 15. 16.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[0. 1. 2.]
 [3. 4. 5.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[ 2.  4.  6.]
 [ 8. 10. 12.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[ 1.  4.  9.]
 [16. 25. 36.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[14. 32.]
 [32. 77.]], shape=(2, 2), dtype=float32)


### tensor & numpy

In [20]:
a = np.array([2., 4., 5.])
print(tf.constant(a),'\n')
print(t.numpy(),'\n')
print(tf.square(a),'\n')
print(np.square(t))

tf.Tensor([2. 4. 5.], shape=(3,), dtype=float32) 

[[1. 2. 3.]
 [4. 5. 6.]] 

tf.Tensor([ 4. 16. 25.], shape=(3,), dtype=float64) 

[[ 1.  4.  9.]
 [16. 25. 36.]]


In [21]:
tf.constant(a, dtype = tf.float32)

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([2., 4., 5.], dtype=float32)>

### type transformation

In [36]:
# tf.constant(2.)+tf.constant(40) # error!

In [37]:
# tf.constant(2.)+tf.constant(40., dtype = tf.float64) # error!

In [28]:
t2 = tf.constant(40., dtype = tf.float64)
tf.constant(2.0) + tf.cast(t2, tf.float32)

<tf.Tensor: shape=(), dtype=float32, numpy=42.0>

### variable

In [29]:
v = tf.Variable([[1.,2.,3.],[4.,5.,6.]])
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [31]:
v.assign(2*v)
print(v)
v[0,1].assign(42)
print(v)
v[:,2].assign([0.,1.])
print(v)
v.scatter_nd_update(indices = [[0,0],[1,2]], updates = [100., 200.])
print(v)

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[ 4., 84., 12.],
       [16., 20., 24.]], dtype=float32)>
<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[ 4., 42., 12.],
       [16., 20., 24.]], dtype=float32)>
<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[ 4., 42.,  0.],
       [16., 20.,  1.]], dtype=float32)>
<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[100.,  42.,   0.],
       [ 16.,  20., 200.]], dtype=float32)>


# user define Model & Algorithm

### user define loss function

In [33]:
# Huber loss (with fixed threshold : 1)
def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss = tf.abs(error) -0.5
    return tf.where(is_small_error, squared_loss, linear_loss)

In [34]:
# data preparing
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target.reshape(-1, 1), random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

In [38]:
# model building
input_shape = X_train.shape[1:]

model = keras.models.Sequential([
    keras.layers.Dense(30, activation="selu", kernel_initializer="lecun_normal",
                       input_shape=input_shape),
    keras.layers.Dense(1),
])

# fitting (using Huber loss)
model.compile(loss=huber_fn, optimizer="nadam", metrics=["mae"])
model.fit(X_train_scaled, y_train, epochs=3,
          validation_data=(X_valid_scaled, y_valid))

Train on 11610 samples, validate on 3870 samples
Epoch 1/3
11610/11610 [==============================] - 2s 209us/sample - loss: 0.6247 - mae: 0.9969 - val_loss: 0.2884 - val_mae: 0.5913
Epoch 2/3
11610/11610 [==============================] - 1s 110us/sample - loss: 0.2193 - mae: 0.5176 - val_loss: 0.2361 - val_mae: 0.5232
Epoch 3/3
11610/11610 [==============================] - 1s 102us/sample - loss: 0.2054 - mae: 0.4980 - val_loss: 0.2242 - val_mae: 0.5086


### saving & loading models with custom elements 

In [39]:
# model save
model.save("my_model_with_a_custom_loss.h5")

In [40]:
# model load
model = keras.models.load_model("my_model_with_a_custom_loss.h5",
                                custom_objects={"huber_fn": huber_fn})

In [41]:
# set threshold 
def create_huber(threshold = 1.0):
    def huber_fn(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < threshold
        squared_loss = tf.square(error) / 2
        linear_loss = threshold * tf.abs(error) - threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    return huber_fn

In [42]:
model.compile(loss = create_huber(2.0), optimizer = "nadam")
model.fit(X_train_scaled, y_train, epochs=3,
          validation_data=(X_valid_scaled, y_valid))

Train on 11610 samples, validate on 3870 samples
Epoch 1/3
11610/11610 [==============================] - 2s 195us/sample - loss: 0.2275 - val_loss: 0.2647
Epoch 2/3
11610/11610 [==============================] - 1s 99us/sample - loss: 0.2226 - val_loss: 0.2457
Epoch 3/3
11610/11610 [==============================] - 1s 99us/sample - loss: 0.2179 - val_loss: 0.2336


In [ ]:
# save & load
# 임계값은 함께 저장되지 않는다.
model.save("my_model_with_a_custom_loss_threshold_2.h5")
model = keras.models.load_model("my_model_with_a_custom_loss_threshold_2.h5",
                                custom_objects={"huber_fn": create_huber(2.0)})

In [ ]:
# 대안으로 아래와 같이 새로운 클래스를 정의하여 임계값까지 저장되도록 할 수 있음
class HuberLoss(keras.losses.Loss):
    def __init__(self, threshold=1.0, **kwargs):
        self.threshold = threshold
        super().__init__(**kwargs)
    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_error = tf.square(error) / 2
        linear_error = self.threshold * tf.abs(error) - self.threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    def get_config(self):
        base_config = super().get_config()
        # 부모 클래스의 config에 새로운 하이퍼파라미터 threshold를 추가하여 반환
        return {**base_config, "threshold":self.threshold}

In [ ]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="selu", kernel_initializer="lecun_normal",
                       input_shape=input_shape),
    keras.layers.Dense(1),
])

model.compile(loss=HuberLoss(2.), optimizer="nadam", metrics=["mae"])
model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))

model.save("my_model_with_a_custom_loss_class.h5")
model = keras.models.load_model("my_model_with_a_custom_loss_class.h5",
                                custom_objects={"HuberLoss": HuberLoss})

#### activation/initializing/regularization/constraint custumizing

In [43]:
def my_softplus(z):
    return tf.math.log(tf.exp(z) + 1.0)

def my_glorot_initializer(shape, dtype = tf.float32):
    stddev = tf.sqrt(2. / (shape[0]+shape[1]))
    return tf.random.normal(shape, stddev = stddev, dtype = dtype)

def my_l1_regularizer(weights):
    return tf.reduce_sum(tf.abs(0.01*weights))

def my_positive_weights(weights):
    return tf.where(weights < 0., tf.zeros_like(weights), weights)

In [ ]:
layer = keras.layers.Dense(1, activation=my_softplus,
                           kernel_initializer=my_glorot_initializer,
                           kernel_regularizer=my_l1_regularizer,
                           kernel_constraint=my_positive_weights)

In [ ]:
class MyL1Regularizer(keras.regularizers.Regularizer):
    def __init__(self, factor):
        self.factor = factor
    def __call__(self, weights):
        return tf.reduce_sum(tf.abs(self.factor * weights))
    def get_config(self):
        return {"factor": self.factor}

In [ ]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="selu", kernel_initializer="lecun_normal",
                       input_shape=input_shape),
    keras.layers.Dense(1, activation=my_softplus,
                       kernel_regularizer=MyL1Regularizer(0.01))
])

### user define metrics

In [ ]:
# streaming metrics
precision = keras.metrics.Precision()

# user define streaming metrics
class HuberMetric(keras.metrics.Mean):
    def __init__(self, threshold=1.0, name='HuberMetric', dtype=None):
        self.threshold = threshold
        self.huber_fn = create_huber(threshold)
        super().__init__(name=name, dtype=dtype)
    def update_state(self, y_true, y_pred, sample_weight=None):
        metric = self.huber_fn(y_true, y_pred)
        super(HuberMetric, self).update_state(metric, sample_weight)
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}        

### user define layer  
####   
동일한 층 블럭이 여러 번 반복되는 네트워크 (a,b,c,a,b,c,a,b,c) 를 만들 경우  
각각의 층 블럭(a,b,c)을 하나의 사용자 정의 층(d)으로 정의하고 (d,d,d)와 같이 만들 수 있다.

In [ ]:
# 가중치/하이퍼파라미터가 필요 없는 사용자 정의 층
# 파이썬 함수를 만든 후 keras.layers.Lambda 층으로 감싼다.
exponential_layer = keras.layers.Lambda(lambda x: tf.exp(x))

In [ ]:
# 상태가 있는 사용자 정의 층